<a href="https://colab.research.google.com/github/juyounLee/DeepLearningZeroToAll_TF2./blob/main/06_02_softmax_animal_classfier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import math

In [2]:
# load the data
xy = np.loadtxt('/content/sample_data/data-04-zoo.csv', delimiter= ',', dtype=np.float32)
x_data = xy[:, :-1]
y_data = xy[:, [-1]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.int32)

In [3]:
# make y_data as one_hot shape
y_data = tf.one_hot(list(y_data), 7) 
y_data = tf.reshape(y_data, [-1, 7])

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

In [5]:
print(x_data.shape)
print(y_data.shape)

(101, 16)
(101, 7)


In [6]:
# initialize the variables
W = tf.Variable(tf.random.normal([16, 7]))
b = tf.Variable(tf.random.normal([7]))

print(W.numpy())

[[-0.7120378   0.77628875  1.112596    0.8115335   0.23963797 -0.52076346
   0.07427558]
 [ 0.22650367  1.2130828  -1.7152549  -0.43047753 -0.6392438  -0.6624469
   0.8196744 ]
 [-0.09425044  0.8666857  -1.6672714   4.0496564  -0.7628796   0.04840175
   1.2660315 ]
 [-1.1925535  -0.53776443  1.358448   -0.8446073  -0.3432678  -0.82670623
   1.8647814 ]
 [-0.5270981   1.1953229  -0.43455172  0.20463017 -1.7230135   0.7742736
   0.11500566]
 [-0.27446225  0.10821293  1.3513514   0.8484344  -0.5787349   0.12900735
   0.32798636]
 [ 1.3859396  -0.6353007   0.03988767  1.7774875  -0.9470912  -1.7416054
   1.0290751 ]
 [ 1.1605405   0.25207672  0.13058601 -1.5434034   1.7146838   1.5707067
  -0.02131092]
 [ 0.8442215   0.43102753  0.17313214  1.0295749  -0.7465863  -0.18633588
   1.0344824 ]
 [ 1.5166847   0.81611186  2.0466194  -0.49472374  0.05862716 -0.00836055
  -1.1195561 ]
 [ 0.37198043  1.1749257  -1.5135643   0.22299306  0.62729144  0.8288711
  -1.3561667 ]
 [-2.0089161   0.07909955 

In [7]:
# logit function
def logit_fn(x):
  logit = tf.matmul(x, W)+b
  return logit

In [8]:
# hypothesis
def hypothesis_fn(x):
  logit = logit_fn(x)
  hypothesis = tf.nn.softmax(logit)
  return hypothesis

In [9]:
# cost function
def cost_fn(x, y):
  hypothesis = hypothesis_fn(x)
  logit = logit_fn(x)
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, logits=logit))
  return cost

In [10]:
# calculate the gradient
def grad(x, y):
  with tf.GradientTape() as tape:
    loss_value = cost_fn(x, y)
  return tape.gradient(loss_value, [W, b])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [11]:
# calculate accuracy
def pred_fn(x, y):
  hypothesis = tf.argmax(hypothesis_fn(x), 1)
  correct_value = tf.math.equal(hypothesis, tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_value, dtype=tf.float32))
  return accuracy

In [12]:
for step in range(2001):
  for feature, label in iter(dataset):
    grads = grad(feature, label)
    optimizer.apply_gradients(grads_and_vars=zip(grads, [W, b]))
    cost = cost_fn(feature, label)
    accuracy = pred_fn(feature, label)

    if step%200 == 0:
      print('step: {} cost: {:.4f} accuracy: {:8.4f}'.format(step, cost.numpy(), accuracy.numpy()))

step: 0 cost: 5.4530 accuracy:   0.1683
step: 200 cost: 1.6956 accuracy:   0.5743
step: 400 cost: 1.1788 accuracy:   0.7426
step: 600 cost: 0.9966 accuracy:   0.8020
step: 800 cost: 0.8831 accuracy:   0.8218
step: 1000 cost: 0.7984 accuracy:   0.8317
step: 1200 cost: 0.7316 accuracy:   0.8416
step: 1400 cost: 0.6771 accuracy:   0.8416
step: 1600 cost: 0.6311 accuracy:   0.8416
step: 1800 cost: 0.5914 accuracy:   0.8515
step: 2000 cost: 0.5566 accuracy:   0.8515


In [13]:
print(x_data[3])
print(y_data[3])

[1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 4. 0. 0. 1.]
tf.Tensor([1. 0. 0. 0. 0. 0. 0.], shape=(7,), dtype=float32)


In [14]:
x_data = [[1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.]]
x_data = np.array(x_data, dtype=np.float32)

y_estimate = hypothesis_fn(x_data)
print(tf.argmax(y_estimate, 1))

tf.Tensor([0], shape=(1,), dtype=int64)
